# SimpleHouseRSla notebook example

In this example, the usage of the model "SimpleHouseSlab-v0" is demonstrated.

First, we import Energym and create the simulation environment by specifying the model, a weather file and the number of simulation days.

In [1]:

import sys
sys.path.insert(0, "/home/lesong/energym/")
import energym

weather = "CH_BS_Basel"
env = energym.make("SimpleHouseRSla-v0", weather=weather, simulation_days=20)

the initial variables are {'u': 0.0}


The control inputs can be inspected using the `get_inputs_names()` method.

In [2]:
inputs = env.get_inputs_names()
print(inputs)
print(env.input_specs)

['u']
{'u': {'type': 'scalar', 'lower_bound': 0, 'upper_bound': 1, 'default': 0, 'description': 'Heat pump power fraction setpoint.'}}


To run the simulation, a number of steps is specified (here 288 steps per day for 10 days), a control input is specified and passed to the simulation model with the `step()` method. To generate some plots later on, we save all the outputs in lists.

In [3]:
from scipy import signal
import random

steps = 288*10
out_list = []
outputs = env.get_output()
controls = []
hour = 0
for i in range(steps):
    control = {}
    # control['u'] = [0.5*(signal.square(0.1*i)+1.0)]
    control['u'] = [random.random()]
    controls +=[ {p:control[p][0] for p in control} ]
    outputs = env.step(control)
    _,hour,_,_ = env.get_date()
    out_list.append(outputs)
    # print(env.get_kpi(start_ind=i, end_ind=i+1))

TypeError: 'float' object is not subscriptable

Since the outputs are given as dictionaries and are collected in lists, we can simply load them as a pandas.DataFrame.

In [ ]:
import pandas as pd
out_df = pd.DataFrame(out_list)

In [ ]:
out_df

,TOut.T,heaPum.COP,heaPum.COPCar,heaPum.P,heaPum.QCon_flow,heaPum.QEva_flow,heaPum.TConAct,heaPum.TEvaAct,preHea.Q_flow,sla.QTot,...,sunRad.y,temRet.T,temRoo.T,temSup.T,weaBus.HDifHor,weaBus.HDirNor,weaBus.HGloHor,weaBus.HHorIR,y,time
0,273.040697,4.845457,10.197181,5000.0,24227.286656,-19227.286656,306.383478,276.337580,0.0,11709.697791,...,0.0,299.165850,293.110455,304.431161,0.0,0.0,0.0,198.356554,293.110455,300.0
1,273.050842,4.497996,9.465954,5000.0,22489.978466,-17489.978466,309.695190,276.978448,0.0,15166.910881,...,0.0,302.821032,293.214613,307.887631,0.0,0.0,0.0,198.314931,293.214613,600.0
2,273.060017,4.309981,9.070281,5000.0,21549.906252,-16549.906252,311.686527,277.323036,0.0,17116.428224,...,0.0,304.992757,293.394047,309.957447,0.0,0.0,0.0,198.276953,293.394047,900.0
3,273.067805,4.200541,8.839966,5000.0,21002.704601,-16002.704601,312.921007,277.522566,0.0,18209.589073,...,0.0,306.333322,293.614137,311.237577,0.0,0.0,0.0,198.244444,293.614137,1200.0
4,273.073788,4.132531,8.696839,5000.0,20662.652743,-15662.652743,313.718667,277.645945,0.0,18812.735803,...,0.0,307.199890,293.855164,312.063525,0.0,0.0,0.0,198.219227,293.855164,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,272.647407,4.177799,8.792107,5000.0,20888.997266,-15888.997266,313.195114,277.572809,0.0,20634.563400,...,0.0,306.536507,291.549997,311.523715,0.0,0.0,0.0,244.300702,291.549997,171600.0
572,272.658750,4.167759,8.770977,5000.0,20838.795940,-15838.795940,313.311141,277.589792,0.0,20425.150407,...,0.0,306.675304,291.872749,311.643573,0.0,0.0,0.0,244.630757,291.872749,171900.0
573,272.677037,4.154181,8.742402,5000.0,20770.904853,-15770.904853,313.469499,277.613279,0.0,20270.970815,...,0.0,306.858999,292.185782,311.807262,0.0,0.0,0.0,244.957719,292.185782,172200.0
574,272.703657,4.138791,8.710015,5000.0,20693.957151,-15693.957151,313.650436,277.640116,0.0,20149.343094,...,0.0,307.066410,292.490609,311.994302,0.0,0.0,0.0,245.280998,292.490609,172500.0


To generate plots, we can directly get the data from the DataFrames, by using the key names. Displayed are the room temperature, the supply temperature and the return temperature, as well as the external temperature, and the heat pump energy.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

f, (ax1,ax2,ax3) = plt.subplots(3,figsize=(10,15))#


ax1.plot(out_df['temRoo.T']-273.15, 'r')
ax1.plot(out_df['temSup.T']-273.15, 'b--')
ax1.plot(out_df['temRet.T']-273.15, 'orange')
ax1.set_ylabel('Temp')
ax1.set_xlabel('Steps')

ax2.plot(out_df['TOut.T']-273.15, 'r')
ax2.set_ylabel('Temp')
ax2.set_xlabel('Steps')

ax3.plot(out_df['heaPum.P'], 'g')
ax3.set_ylabel('Energy')
ax3.set_xlabel('Steps')

plt.subplots_adjust(hspace=0.4)

plt.show()

<IPython.core.display.Javascript object>

To end the simulation, the `close()` method is called. It deletes files that were produced during the simulation and stores some information about the simulation in the *energym_runs* folder.

In [ ]:
env.close()